In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
import json 
import random as rd

The approach of Bertsimas and Periakis to dynamic pricing consists of three steps.
- Demand estimation
- Estimation of the competitor's price
- (Myopic) own price setting

The demand model is
$$
d_{k,t} = \beta^{0}_{k,t} +  \beta^{1}_{k,t} p_{1,t} +  \beta^{2}_{k,t} p_{2,t} + \epsilon_{k,t}
$$
where $k=1$ refers to you and $k=2$ refers to your competitor.

The input to the demand model is
- your own prices up to sales period t-1
- the quantity of demand you sold at you price levelt up to period t-1
- your competitor's prices up to sales period t-1

We want to minimize the deviation from the observed values to the estimated values given the demand model.

$$ 
\min \sum\limits_{\tau = 1}^{t-1} \left|d_{1,\tau} - \left( \hat{\beta}^{0}_{1,\tau} +  \hat{\beta}^{1}_{1,\tau} p_{1,\tau} +  \hat{\beta}^{2}_{1,\tau} p_{2,\tau} \right) \right| 
$$
such that $|\hat{\beta}^i_{1,\tau} - \hat{\beta}^i_{1,\tau+1}| \le \delta_1(i)$ for $i = 0, 1, 2$ and $\tau = 1, \ldots, t-2$

This nonlinear optimization problem can be linearized as follows:

$$ \min \sum\limits_{\tau = 1}^{t-1} \Delta_{\tau} $$
such that 
$
\Delta_{\tau} \ge \left|d_{1,\tau} - \left( \hat{\beta}^{0}_{1,\tau} +  \hat{\beta}^{1}_{1,\tau} p_{1,\tau} +  \hat{\beta}^{2}_{1,\tau} p_{2,\tau} \right) \right|
$ for all $\tau = 1, \ldots, t-1$ 
and
$|\hat{\beta}^i_{1,\tau} - \hat{\beta}^i_{1,\tau+1}| \le \delta_1(i)$ for $i = 0, 1, 2$ and $\tau = 1, \ldots, t-2$

The first constraint can be linearized by considering the positive and negative part of the absolute value:

$$ 
\Delta_{\tau} \ge d_{1,\tau} - \left( \hat{\beta}^{0}_{1,\tau} +  \hat{\beta}^{1}_{1,\tau} p_{1,\tau} +  \hat{\beta}^{2}_{1,\tau} p_{2,\tau} \right)
$$ and
$$ 
\Delta_{\tau} \ge -d_{1,\tau} + \left( \hat{\beta}^{0}_{1,\tau} +  \hat{\beta}^{1}_{1,\tau} p_{1,\tau} +  \hat{\beta}^{2}_{1,\tau} p_{2,\tau} \right)
$$

The same holds for the second constraint, i.e.
$$
\hat{\beta}^i_{1,\tau} - \hat{\beta}^i_{1,\tau+1} \le \delta_1(i)
$$ 
and
$$
-\hat{\beta}^i_{1,\tau} + \hat{\beta}^i_{1,\tau+1}| \le \delta_1(i)
$$

# Load data

### Load data from one competition

In [9]:
df = pd.read_csv("../../Results/11.24/duopoly_competition_details.csv")
df = df[(df['selling_season'] == df['selling_season'][0]) & (df['competition_id'] == df['competition_id'][0])]
X = df[[ 'price','price_competitor']]
y = df[['demand']] 

X = X.to_numpy()
y = np.asarray(y).reshape(-1)
# plt.plot(X[0:], y)
print(X, y)

[[31.2 74.8]
 [29.6 14. ]
 [28.1 60. ]
 [26.7 31.7]
 [45.1 73. ]
 [42.8 86.6]
 [40.7 80. ]
 [38.7 27.9]
 [36.7  8.1]
 [55.1  0.7]
 [52.4 65. ]
 [49.8 15.5]
 [47.3 53. ]
 [44.9 40.6]
 [35.  49.3]
 [33.2 28.9]
 [31.6 35.7]
 [30.  96.5]
 [28.5 61.8]
 [54.5 93.2]
 [51.7 29.5]
 [49.1  7.8]
 [46.7 18.7]
 [44.4 66.2]
 [43.1 49.6]
 [40.9 45.8]
 [38.9 64.8]
 [36.9 54.8]
 [35.1 92.3]
 [61.4 77.9]
 [58.4 90.3]
 [55.4 17. ]
 [52.7 70.4]
 [50.  28.1]
 [56.7 23.4]
 [53.9 16.6]
 [51.2 31.1]
 [48.6 41.5]
 [46.2 38.7]
 [30.3 69.5]
 [28.8 25.5]
 [27.3 66.6]
 [26.   0.4]
 [24.7 24.4]
 [37.3 22.4]
 [35.4 27.2]
 [33.6 40.4]
 [32.  40.5]
 [30.4 67.8]
 [39.7 82.4]
 [37.7 53.5]
 [35.8 72.5]
 [34.  92.6]
 [32.3 23.6]
 [64.9 72.8]
 [61.7 50.4]
 [58.6 46.7]
 [55.7 18.5]
 [52.9 32.5]
 [44.2 98.6]
 [42.   7.3]
 [39.9 40.7]
 [37.9 76.1]
 [36.  77. ]
 [59.9  4.8]
 [56.9 61.3]
 [54.1 79.9]
 [51.4 54.8]
 [48.8 71.5]
 [54.5 44. ]
 [51.8 33.6]
 [49.2 66.9]
 [46.7 35.5]
 [44.4 48.1]
 [45.6 99.2]
 [43.3 26.4]
 [41.2 53.8]

### Load data from all competition

On doit parcourir tout le fichier ../../Results/ en extraire chaque date. Dans chaque de ces fichiers on trouvera un `duopoly_competition_details.csv`, que l'on extrairera. Pour garder cela cohérent, on conservera la colonne competition. 

In [10]:
all_df = pd.DataFrame()
back_path = "../../Results"
following_path = "duopoly_competition_details.csv"
for folder in os.listdir(back_path):
    df_path = back_path + '/' + folder + '/' + following_path
    df = pd.read_csv(df_path)
    all_df = pd.concat([all_df, df[['demand','price','price_competitor','competition_id']]])

# print(all_df['competition_id'].nunique())

print(all_df[all_df['demand'] == all_df['demand'].max()])
all_df


       demand  price  price_competitor competition_id
10743      10   24.6              31.4         HTmLPU


,demand,price,price_competitor,competition_id
0,0,150.0,30.0,35gvEP
1,0,150.0,80.0,35gvEP
2,0,150.0,40.0,35gvEP
3,0,150.0,90.0,35gvEP
4,0,150.0,89.7,35gvEP
...,...,...,...,...
29995,0,20.0,4.4,fw6wpA
29996,0,20.0,2.6,fw6wpA
29997,0,20.0,1.3,fw6wpA
29998,0,20.0,0.5,fw6wpA


# Regression 

## On one competition

### OLS

In [11]:
solver = 'ipopt'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

In [12]:
def ols_regression(X, y):
    m = pyo.ConcreteModel("OLS Regression")

    n, k = X.shape

    # note use of Python style zero based indexing
    m.I = pyo.RangeSet(0, n - 1)
    m.J = pyo.RangeSet(0, k - 1)

    m.e = pyo.Var(m.I, domain=pyo.Reals)
    m.m = pyo.Var(m.J)
    m.b = pyo.Var()

    @m.Constraint(m.I)
    def residuals(m, i):
        return m.e[i] == y[i] - sum(X[i][j] * m.m[j] for j in m.J) - m.b

    @m.Objective(sense=pyo.minimize)
    def sum_of_square_errors(m):
        return sum((m.e[i]) ** 2 for i in m.I)

    return m


m = ols_regression(X, y)
SOLVER.solve(m)
m.m.display()
m.b.display()


m : Size=2, Index=J
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None : -0.017455268055988963 :  None : False : False :  Reals
      1 :  None :  0.004841880918439343 :  None : False : False :  Reals
b : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 0.7128910769609748 :  None : False : False :  Reals


In [6]:
def compute_regression_metrics(X, y, coef, intercept):
    # Prédictions
    y_pred = X @ coef + intercept
    
    # Résidus
    residuals = y - y_pred
    
    # Métriques standard
    n = len(y)
    p = X.shape[1]

    SSE = np.sum((y - y_pred)**2)
    SST = np.sum((y - np.mean(y))**2)
    SSR = SST - SSE

    R2 = 1 - SSE / SST
    R2_adj = 1 - (SSE/(n - p - 1)) / (SST/(n - 1))

    MSE = SSE / n
    RMSE = np.sqrt(MSE)
    MAE = np.mean(np.abs(residuals))

    return {
        "R2": R2,
        "R2_adj": R2_adj,
        "SSE": SSE,
        "SSR": SSR,
        "SST": SST,
        "MSE": MSE,
        "RMSE": RMSE,
        "MAE": MAE,
        "residuals": residuals,
        "y_pred": y_pred
    }


## One all competition

Pour chaque competition, on va faire une regression linéaire. On la sauvegarde dans un dictionnaire

In [7]:
import tqdm

reg_lin_results = {}
for comp in tqdm.tqdm(all_df['competition_id'].unique()): 
    X = all_df[all_df['competition_id'] == comp][['price', 'price_competitor']]
    y = all_df[all_df['competition_id'] == comp][['demand']]
    X = X.to_numpy()
    y = np.asarray(y).reshape(-1)
    if sum(y) < 20 :
        print(f"On {comp} competition, we did'nt sale any shit")
        continue

    m = ols_regression(X, y)
    SOLVER.solve(m)
    coef = np.array([m.m[k].value for k in m.m])
    intercept = m.b.value

    metrics = compute_regression_metrics(X, y, coef, intercept)
    reg_lin_results[comp] = {
        "coeff": [float(m.m[k].value) for k in m.m],
        "intercept": float(m.b.value),
        "R2" : metrics['R2'],
        "MSE": metrics['MSE']
    }

dict_folder = './LinReg_results'
os.makedirs(dict_folder, exist_ok = True)
with open(os.path.join(dict_folder, 'dict'), 'w') as f:
    json.dump(reg_lin_results, f)

  0%|          | 0/66 [00:00<?, ?it/s]

On 35gvEP competition, we did'nt sale any shit
On 3dncK4 competition, we did'nt sale any shit


  6%|▌         | 4/66 [00:00<00:03, 16.51it/s]

On 4AnUch competition, we did'nt sale any shit
On 4PKfqE competition, we did'nt sale any shit
On 4SZJPv competition, we did'nt sale any shit


 12%|█▏        | 8/66 [00:02<00:18,  3.16it/s]

...

On 34FWTM competition, we did'nt sale any shit
On 3SLntX competition, we did'nt sale any shit


 24%|██▍       | 16/66 [00:06<00:26,  1.86it/s]

On 4R8bN7 competition, we did'nt sale any shit
On 32gZbC competition, we did'nt sale any shit


 26%|██▌       | 17/66 [00:09<00:26,  1.85it/s]


KeyboardInterrupt: 

Now I have a dictionnary with a linear regression on all my competition where I sold more than 20 items. 

Now I want to predict the demand from my competitor. Today, we will just take the last price. Then, with this equation of the demanding model: 
$$
d_{k,t} = \beta^{0}_{k,t} +  \beta^{1}_{k,t} p_{1,t} +  \beta^{2}_{k,t} p_{2,t} + \epsilon_{k,t}
$$
where $k=1$ refers to you and $k=2$ refers to your competitor.

It means that our price is : 
$$
p_{1,t} = \frac{1}{\beta^{1}_{k,t}}
\left( d_{k,t} - \beta^{0}_{k,t} - \beta^{2}_{k,t} p_{2,t} - \epsilon_{k,t} \right).
$$

We are able to compute our own price.

In [13]:
def set_our_price(d, intercept, coeff, last_price_competitor) : 
    return 1/coeff[0] * (d - intercept - last_price_competitor*coeff[1])

We can try it on a single season and see.

In [15]:
with open('LinReg_results/dict') as f:
    dict_linreg = json.load(f) 

rd.seed(42)
competition_observed = rd.choice(list(dict_linreg.keys()))
print(competition_observed)

coeff     = dict_linreg[competition_observed]['coeff']
intercept = dict_linreg[competition_observed]['intercept']
print(coeff, intercept)

3cuFyA
[-0.013864887637616953, 0.013520663949032065] 0.2973779827744229
